In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isna().sum() #thats a lot of missing values :(

In [ ]:
df.isna().sum().sum() #we cannot drop them since we would lose a lot of data

In [ ]:
#attempt to preprocess
cat_columns = df.select_dtypes(include='object').columns #these are the categorical columns
cat_columns

In [ ]:
for x in cat_columns:  #getting count of unique values
    print(x+"  "+str(df[x].nunique()))

In [ ]:
df.columns

In [ ]:
predictors = df[['RainToday', 'RainTomorrow']]
predictors.head()

In [ ]:
#preprocessing stuff
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()  #label encoding only those two variables
for cols in predictors.columns:
    df[cols] = le.fit_transform(df[cols])


In [ ]:
df.head()

In [ ]:
df['Date'].sample(10)  # it seems to be in the format year/month/date

In [ ]:
list = []
for x in df['Date']:
    list.append(len(x))
#print(list)
s = set(list)
print(s)  #we can now confirm that we have the dates in same format with 10 chars
    
    

In [ ]:
df[["year", "month", "date"]] =[x.split("-") for x in df["Date"]] #splitting on'-'

In [ ]:
df.head()

In [ ]:
df[["year", "month", "date"]] = df[["year", "month", "date"]].astype(int)

In [ ]:
df = df.drop(['Date','year'],axis=1)
df.head()

In [ ]:
cat_columns = df.select_dtypes(include='object').columns #now we have only 4 features
cat_columns


In [ ]:
dummies_df = pd.get_dummies(df,columns=cat_columns,drop_first=True)
dummies_df.head()

In [ ]:
dummies_df.columns.tolist()  #dummies worked yaay ^^

In [ ]:
numerical_cols = df.select_dtypes(include=['float64','int64']).columns #getting numerical columns
numerical_cols


In [ ]:
df[numerical_cols].head()


In [ ]:
from sklearn.impute import SimpleImputer  #to fill in some NA values


In [ ]:
df[['month','date']].isna().sum()  #they have no NA values

In [ ]:
impute = SimpleImputer(strategy='mean') #replacing the misssing values with mean
imputed_df = pd.DataFrame(impute.fit_transform(df[numerical_cols]))


In [ ]:
imputed_df.columns = numerical_cols

In [ ]:
imputed_df.head()

In [ ]:
imputed_df[['RainToday','RainTomorrow','month','date']] = imputed_df[['RainToday','RainTomorrow','month','date']].astype(int)

In [ ]:
imputed_df.head()  #now they look better

In [ ]:
dummies_df = dummies_df.drop(numerical_cols,axis=1) #dropping the features already preprocessed in imputed_df
dummies_df.head()

In [ ]:
#now to combine both
new_df = pd.concat([dummies_df,imputed_df],axis=1)
new_df.head()

In [ ]:
new_df.isna().sum().sum()  #ahhh finally we are done .

In [ ]:
#thanks for stopping by :)